In [19]:
import pandas as pd
import numpy as np

cc = pd.read_csv("../data/creditcard.csv")
cc = cc.drop(["Class"], axis=1)
#cc = cc.drop(["Class", "Amount"], axis = 1)
cc.tail()
#cc.Time = np.floor(cc.index / 2848)
cc.Time = np.floor(cc['Time'] / (60*60))

In [20]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
connections.connect("default", host="95.216.186.23", port="19530")

In [21]:


transaction_id = FieldSchema(
  name="transaction_id",
  dtype=DataType.INT64,
  is_primary=True,
)
# transaction_amount = FieldSchema(
#   name="transaction_amount",
#   dtype=DataType.FLOAT_VECTOR
# )
transaction_features = FieldSchema(
  name="transaction_features",
  dtype=DataType.FLOAT_VECTOR,
  dim=28
)
schema = CollectionSchema(
  fields=[transaction_id, transaction_features],
  description="Transaction Similarity Search"
)
collection_name = "transactions"

In [23]:
collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2,

    consistency_level="Strong"
)

In [24]:
utility.has_collection("transactions")
#utility.drop_collection("transactions")
utility.list_collections()

collection = Collection("transactions")
collection.load()

collection.description
collection.indexes


[]

In [9]:
# index = {
#   "metric_type":"IP",
#   "index_type":"HNSW",
#   "params":{"M":16, "efConstruction":500}
# }
#
# collection.create_index("transaction_features", index)

#search_params = {"metric_type": "IP", "params": {"ef": 10}}

# Define loop and data

In [25]:
from sklearn.preprocessing import StandardScaler
import time
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import gc


pd.options.mode.chained_assignment = None

start_time = time.time()

#create empty lists to fill later
# ids_10 = []
# dists_10 = []
ids_50 = []
dists_50 = []

index = {
  "metric_type":"L2",
  "index_type":"HNSW",
  "params":{"M":16, "efConstruction":500}
}
search_params = {"metric_type": "L2", "params": {"ef": 64}}

collection.create_index("transaction_features", index)

for t in tqdm_notebook(range(0, 48)):

    transaction_id = FieldSchema(
    name="transaction_id",
    dtype=DataType.INT64,
    is_primary=True,
    )
    # transaction_amount = FieldSchema(
    #   name="transaction_amount",
    #   dtype=DataType.FLOAT_VECTOR
    # )
    transaction_features = FieldSchema(
      name="transaction_features",
      dtype=DataType.FLOAT_VECTOR,
      dim=28
    )
    schema = CollectionSchema(
      fields=[transaction_id, transaction_features],
      description="Transaction Similarity Search"
    )
    collection_name = "transactions"





    collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2,

    consistency_level="Strong"
    )




    utility.has_collection("transactions")
    #utility.drop_collection("transactions")
    utility.list_collections()

    collection = Collection("transactions")
    collection.load()

    collection.description
    collection.indexes












    #time.sleep(15)
    start2 = time.time()
    #filter data based on time
    data = cc.loc[cc.Time <= t]

    #use standardscalar on data
    #data.iloc[:, 1:29] = StandardScaler().fit_transform(data.iloc[:, 1:29])

    ######Define data for milvus
    data_list = [data.index.tolist(), data.iloc[:, 1:29].values.tolist()]
    collection = Collection("transactions")
    mr = collection.insert(data_list)

    #create time index
    start = cc.Time.searchsorted(t, side='left')




    #50 !!!!
    #####search collection from most alike features form milvus
    # search_params = {"metric_type": "IP", "params": {"ef": 500}}
    results_50 = collection.search(
        data=data.iloc[:, 1:29].values.tolist(),
        anns_field="transaction_features",
        param=search_params,
        limit=50,
        expr=None,
        consistency_level="Strong"
    )

    #Append only the new results
    for i in range(start, len(results_50)):
        ids_50.append(results_50[i].ids)
        dists_50.append(results_50[i].distances)



    # #256 #not used, too large
    # search_params = {"metric_type": "L2", "params": {"nprobe": 256}}
    # results_256 = collection.search(
    #     data=data.iloc[:, 1:29].values.tolist(),
    #     anns_field="transaction_features",
    #     param=search_params,
    #     limit=256,
    #     expr=None,
    #     consistency_level="Strong"
    # )
    #
    # #Append only the new results
    # for i in range(start, len(results_256)):
    #     ids_256.append(results_256[i].ids)
    #     dists_256.append(results_256[i].distances)
    
    
    #manually delete things and call garbage collector, speed ups process significantly
    del data
    del data_list
    del results_50
    gc.collect()



    utility.drop_collection("transactions")



    end2 = time.time()
    print(f"iteration {t} took: {end2 - start2}")

end = time.time()
print(f"Runtime: {end - start_time}")

In [28]:
df_2 = pd.DataFrame(list(zip(ids_50, dists_50)),
               columns =['name_50', 'dist_50'])

df_2.to_csv('../data/hnsw_server.csv')

In [27]:
df_2.tail()

,name_50,dist_50
284802,"[284802, 283911, 283158, 282467, 282496, 28245...","[0.0, 1.3466992378234863, 5.384416580200195, 8..."
284803,"[284803, 267639, 183512, 188104, 255345, 24610...","[0.0, 1.6716729402542114, 2.555665969848633, 3..."
284804,"[284804, 206752, 272604, 222084, 257755, 22623...","[0.0, 2.5649635517766e-06, 3.142173227388412e-..."
284805,"[284805, 262976, 278322, 228114, 282160, 25468...","[0.0, 1.4927024841308594, 1.96480393409729, 2...."
284806,"[284806, 232882, 278599, 196901, 242015, 19700...","[0.0, 2.449662208557129, 3.0682601928710938, 3..."
